# KubeFlow Pipeline Using TFX OSS Components

In this notebook, we will demo: 

* Defining a KubeFlow pipeline with Python DSL
* Submiting it to Pipelines System
* Customize a step in the pipeline

We will use a pipeline that includes some TFX OSS components such as [TFDV](https://github.com/tensorflow/data-validation), [TFT](https://github.com/tensorflow/transform), [TFMA](https://github.com/tensorflow/model-analysis).

## Setup

In [1]:
# Set your output and project. !!!Must Do before you can proceed!!!
EXPERIMENT_NAME = 'demo'
OUTPUT_DIR = 'Your-Gcs-Path' # Such as gs://bucket/objact/path
PROJECT_NAME = 'Your-Gcp-Project-Name'
BASE_IMAGE='gcr.io/%s/pusherbase:dev' % PROJECT_NAME
TARGET_IMAGE='gcr.io/%s/pusher:dev' % PROJECT_NAME
TARGET_IMAGE_TWO='gcr.io/%s/pusher_two:dev' % PROJECT_NAME
KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.16/kfp.tar.gz'
TRAIN_DATA = 'gs://ml-pipeline-playground/tfx/taxi-cab-classification/train.csv'
EVAL_DATA = 'gs://ml-pipeline-playground/tfx/taxi-cab-classification/eval.csv'
HIDDEN_LAYER_SIZE = '1500'
STEPS = 3000
DATAFLOW_TFDV_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tfdv:ac833a084b32324b56ca56e9109e05cde02816a4'
DATAFLOW_TFT_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tft:ac833a084b32324b56ca56e9109e05cde02816a4'
DATAFLOW_TFMA_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tfma:ac833a084b32324b56ca56e9109e05cde02816a4'
DATAFLOW_TF_PREDICT_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tf-predict:ac833a084b32324b56ca56e9109e05cde02816a4'
KUBEFLOW_TF_TRAINER_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-tf-trainer:ac833a084b32324b56ca56e9109e05cde02816a4'
KUBEFLOW_TF_TRAINER_GPU_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-tf-trainer-gpu:ac833a084b32324b56ca56e9109e05cde02816a4'
KUBEFLOW_DEPLOYER_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-deployer:ac833a084b32324b56ca56e9109e05cde02816a4'
DEPLOYER_MODEL = 'notebook_tfx_taxi'
DEPLOYER_VERSION_DEV = 'dev'
DEPLOYER_VERSION_PROD = 'prod'
DEPLOYER_VERSION_PROD_TWO = 'prodtwo'

In [ ]:
# Install Pipeline SDK
!pip3 install $KFP_PACKAGE --upgrade

## Create an Experiment in the Pipeline System

Pipeline system requires an "Experiment" to group pipeline runs. You can create a new experiment, or call client.list_experiments() to get existing ones.

In [ ]:
# Note that this notebook should be running in JupyterHub in the same cluster as the pipeline system.
# Otherwise it will fail to talk to the pipeline system.
import kfp
from kfp import compiler
import kfp.dsl as dsl
import kfp.notebook
import kfp.gcp as gcp

# If you are using Kubeflow JupyterHub, then no need to set host in Client() constructor.
# But if you are using your local Jupyter instance, and have a kubectl connection to the cluster,
# Then do:
#     client = kfp.Client('127.0.0.1:8080/pipeline')
client = kfp.Client()
exp = client.create_experiment(name=EXPERIMENT_NAME)

## Test Run a Pipeline

In [ ]:
# Download a pipeline package
!gsutil cp gs://ml-pipeline-playground/coin.tar.gz .

In [ ]:
run = client.run_pipeline(exp.id, 'coin', 'coin.tar.gz')

## Define a Pipeline

Authoring a pipeline is just like authoring a normal Python function. The pipeline function describes the topology of the pipeline. Each step in the pipeline is typically a ContainerOp --- a simple class or function describing how to interact with a docker container image. In the below pipeline, all the container images referenced in the pipeline are already built. The pipeline starts with a TFDV step which is used to infer the schema of the data. Then it uses TFT to transform the data for training. After a single node training step, it analyze the test data predictions and generate a feature slice metrics view using a TFMA component. At last, it deploys the model to TF-Serving inside the same cluster.

In [ ]:
import kfp.dsl as dsl


# Below are a list of helper functions to wrap the components to provide a simpler interface for pipeline function.
def dataflow_tf_data_validation_op(inference_data: 'GcsUri', validation_data: 'GcsUri', column_names: 'GcsUri[text/json]', key_columns, project: 'GcpProject', mode, validation_output: 'GcsUri[Directory]', step_name='validation'):
    return dsl.ContainerOp(
        name = step_name,
        image = DATAFLOW_TFDV_IMAGE,
        arguments = [
            '--csv-data-for-inference', inference_data,
            '--csv-data-to-validate', validation_data,
            '--column-names', column_names,
            '--key-columns', key_columns,
            '--project', project,
            '--mode', mode,
            '--output', validation_output,
        ],
        file_outputs = {
            'schema': '/schema.txt',
        }
    )

def dataflow_tf_transform_op(train_data: 'GcsUri', evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', project: 'GcpProject', preprocess_mode, preprocess_module: 'GcsUri[text/code/python]', transform_output: 'GcsUri[Directory]', step_name='preprocess'):
    return dsl.ContainerOp(
        name = step_name,
        image = DATAFLOW_TFT_IMAGE,
        arguments = [
            '--train', train_data,
            '--eval', evaluation_data,
            '--schema', schema,
            '--project', project,
            '--mode', preprocess_mode,
            '--preprocessing-module', preprocess_module,
            '--output', transform_output,
        ],
        file_outputs = {'transformed': '/output.txt'}
    )


def tf_train_op(transformed_data_dir, schema: 'GcsUri[text/json]', learning_rate: float, hidden_layer_size: int, steps: int, target: str, preprocess_module: 'GcsUri[text/code/python]', training_output: 'GcsUri[Directory]', step_name='training', use_gpu=False):
    tf_train_op = dsl.ContainerOp(
        name = step_name,
        image = KUBEFLOW_TF_TRAINER_IMAGE,
        arguments = [
            '--transformed-data-dir', transformed_data_dir,
            '--schema', schema,
            '--learning-rate', learning_rate,
            '--hidden-layer-size', hidden_layer_size,
            '--steps', steps,
            '--target', target,
            '--preprocessing-module', preprocess_module,
            '--job-dir', training_output,
        ],
        file_outputs = {'train': '/output.txt'}
    )
    if use_gpu:
        tf_train_op.image = KUBEFLOW_TF_TRAINER_GPU_IMAGE
        tf_train_op.set_gpu_limit(1)
    
    return tf_train_op

def dataflow_tf_model_analyze_op(model: 'TensorFlow model', evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', project: 'GcpProject', analyze_mode, analyze_slice_column, analysis_output: 'GcsUri', step_name='analysis'):
    return dsl.ContainerOp(
        name = step_name,
        image = DATAFLOW_TFMA_IMAGE,
        arguments = [
            '--model', model,
            '--eval', evaluation_data,
            '--schema', schema,
            '--project', project,
            '--mode', analyze_mode,
            '--slice-columns', analyze_slice_column,
            '--output', analysis_output,
        ],
        file_outputs = {'analysis': '/output.txt'}
    )


def dataflow_tf_predict_op(evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', target: str, model: 'TensorFlow model', predict_mode, project: 'GcpProject', prediction_output: 'GcsUri', step_name='prediction'):
    return dsl.ContainerOp(
        name = step_name,
        image = DATAFLOW_TF_PREDICT_IMAGE,
        arguments = [
            '--data', evaluation_data,
            '--schema', schema,
            '--target', target,
            '--model',  model,
            '--mode', predict_mode,
            '--project', project,
            '--output', prediction_output,
        ],
        file_outputs = {'prediction': '/output.txt'}
    )

def kubeflow_deploy_op(model: 'TensorFlow model', tf_server_name, step_name='deploy'):
    return dsl.ContainerOp(
        name = step_name,
        image = KUBEFLOW_DEPLOYER_IMAGE,
        arguments = [
            '--model-export-path', '%s/export/export' % model,
            '--server-name', tf_server_name
        ]
    )


# The pipeline definition
@dsl.pipeline(
  name='TFX Taxi Cab Classification Pipeline Example',
  description='Example pipeline that does classification with model analysis based on a public BigQuery dataset.'
)
def taxi_cab_classification(
    output,
    project,
    column_names=dsl.PipelineParam(name='column-names', value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/column-names.json'),
    key_columns=dsl.PipelineParam(name='key-columns', value='trip_start_timestamp'),
    train=dsl.PipelineParam(name='train', value=TRAIN_DATA),
    evaluation=dsl.PipelineParam(name='evaluation', value=EVAL_DATA),
    validation_mode=dsl.PipelineParam(name='validation-mode', value='local'),
    preprocess_mode=dsl.PipelineParam(name='preprocess-mode', value='local'),
    preprocess_module: dsl.PipelineParam=dsl.PipelineParam(name='preprocess-module', value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/preprocessing.py'),
    target=dsl.PipelineParam(name='target', value='tips'),
    learning_rate=dsl.PipelineParam(name='learning-rate', value=0.1),
    hidden_layer_size=dsl.PipelineParam(name='hidden-layer-size', value=HIDDEN_LAYER_SIZE),
    steps=dsl.PipelineParam(name='steps', value=STEPS),
    predict_mode=dsl.PipelineParam(name='predict-mode', value='local'),
    analyze_mode=dsl.PipelineParam(name='analyze-mode', value='local'),
    analyze_slice_column=dsl.PipelineParam(name='analyze-slice-column', value='trip_start_hour')):

    # set the flag to use GPU trainer
    use_gpu = False
    
    validation_output = '%s/{{workflow.name}}/validation' % output
    transform_output = '%s/{{workflow.name}}/transformed' % output
    training_output = '%s/{{workflow.name}}/train' % output
    analysis_output = '%s/{{workflow.name}}/analysis' % output
    prediction_output = '%s/{{workflow.name}}/predict' % output
    tf_server_name = 'taxi-cab-classification-model-{{workflow.name}}'

    validation = dataflow_tf_data_validation_op(train, evaluation, column_names, key_columns, project, validation_mode, validation_output).apply(gcp.use_gcp_secret('user-gcp-sa'))

    preprocess = dataflow_tf_transform_op(train, evaluation, validation.outputs['schema'], project, preprocess_mode, preprocess_module, transform_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    training = tf_train_op(preprocess.output, validation.outputs['schema'], learning_rate, hidden_layer_size, steps, target, preprocess_module, training_output, use_gpu=use_gpu).apply(gcp.use_gcp_secret('user-gcp-sa'))
    analysis = dataflow_tf_model_analyze_op(training.output, evaluation, validation.outputs['schema'], project, analyze_mode, analyze_slice_column, analysis_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    prediction = dataflow_tf_predict_op(evaluation, validation.outputs['schema'], target, training.output, predict_mode, project, prediction_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    deploy = kubeflow_deploy_op(training.output, tf_server_name).apply(gcp.use_gcp_secret('user-gcp-sa'))

## Submit the run

In [ ]:
# Compile it into a tar package.
compiler.Compiler().compile(taxi_cab_classification,  'tfx.zip')

# Submit a run.
run = client.run_pipeline(exp.id, 'tfx', 'tfx.zip',
                          params={'output': OUTPUT_DIR,
                                  'project': PROJECT_NAME})

## Customize a step in the above pipeline

Let's say I got the pipeline source code from github, and I want to modify the pipeline a little bit by swapping the last deployer step with my own deployer. Instead of tf-serving deployer, I want to deploy it to Cloud ML Engine service.

### Create and test a python function for the new deployer

In [ ]:
# in order to run it locally we need a python package
!pip3 install google-api-python-client

In [ ]:
@dsl.python_component(
    name='cmle_deployer',
    description='deploys a model to GCP CMLE',
    base_image=BASE_IMAGE
)
def deploy_model(model_name: str, version_name: str, model_path: str, gcp_project: str, runtime: str):

    from googleapiclient import discovery
    from tensorflow.python.lib.io import file_io
    import os
    
    model_path = file_io.get_matching_files(os.path.join(model_path, 'export', 'export', '*'))[0]
    api = discovery.build('ml', 'v1')
    body = {'name': model_name}
    parent = 'projects/%s' % gcp_project
    try:
        api.projects().models().create(body=body, parent=parent).execute()
    except Exception as e:
        # If the error is to create an already existing model. Ignore it.
        print(str(e))
        pass

    import time

    body = {
        'name': version_name,
        'deployment_uri': model_path,
        'runtime_version': runtime
    }

    full_mode_name = 'projects/%s/models/%s' % (gcp_project, model_name)
    response = api.projects().models().versions().create(body=body, parent=full_mode_name).execute()
    
    while True:
        response = api.projects().operations().get(name=response['name']).execute()
        if 'done' not in response or response['done'] is not True:
            time.sleep(10)
            print('still deploying...')
        else:
            if 'error' in response:
                print(response['error'])
            else:
                print('Done.')
            break

In [11]:
# Test the function and make sure it works.
path = 'gs://ml-pipeline-playground/sampledata/taxi/train'
deploy_model(DEPLOYER_MODEL, DEPLOYER_VERSION_DEV, path, PROJECT_NAME, '1.9')

still deploying...
still deploying...
still deploying...
still deploying...
still deploying...
still deploying...
still deploying...
still deploying...
still deploying...
still deploying...
still deploying...
Done.


### Build a Pipeline Step With the Above Function(Note: run either of the two options below)
#### Option One: Specify the dependency directly
Now that we've tested the function locally, we want to build a component that can run as a step in the pipeline. 

In [ ]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
DeployerOp = compiler.build_python_component(
    component_func=deploy_model,
    staging_gcs_path=OUTPUT_DIR,
    dependency=[kfp.compiler.VersionedDependency(name='google-api-python-client', version='1.7.0')],
    base_image='tensorflow/tensorflow:1.12.0-py3',
    target_image=TARGET_IMAGE)

#### Option Two: build a base docker container image with both tensorflow and google api client packages

In [ ]:
%%docker {BASE_IMAGE} {OUTPUT_DIR}
FROM tensorflow/tensorflow:1.10.0-py3
RUN pip3 install google-api-python-client

Once the base docker container image is built, we can build a "target" container image that is base_image plus the python function as entry point. The target container image can be used as a step in a pipeline.

In [ ]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
DeployerOp = compiler.build_python_component(
    component_func=deploy_model,
    staging_gcs_path=OUTPUT_DIR,
    target_image=TARGET_IMAGE)

### Modify the pipeline with the new deployer

In [ ]:
# My New Pipeline. It's almost the same as the original one with the last step deployer replaced.
@dsl.pipeline(
  name='TFX Taxi Cab Classification Pipeline Example',
  description='Example pipeline that does classification with model analysis based on a public BigQuery dataset.'
)
def my_taxi_cab_classification(
    output,
    project,
    model,
    version,
    column_names=dsl.PipelineParam(
        name='column-names',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/column-names.json'),
    key_columns=dsl.PipelineParam(name='key-columns', value='trip_start_timestamp'),
    train=dsl.PipelineParam(
        name='train',
        value=TRAIN_DATA),
    evaluation=dsl.PipelineParam(
        name='evaluation',
        value=EVAL_DATA),
    validation_mode=dsl.PipelineParam(name='validation-mode', value='local'),
    preprocess_mode=dsl.PipelineParam(name='preprocess-mode', value='local'),
    preprocess_module: dsl.PipelineParam=dsl.PipelineParam(
        name='preprocess-module',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/preprocessing.py'),
    target=dsl.PipelineParam(name='target', value='tips'),
    learning_rate=dsl.PipelineParam(name='learning-rate', value=0.1),
    hidden_layer_size=dsl.PipelineParam(name='hidden-layer-size', value=HIDDEN_LAYER_SIZE),
    steps=dsl.PipelineParam(name='steps', value=STEPS),
    predict_mode=dsl.PipelineParam(name='predict-mode', value='local'),
    analyze_mode=dsl.PipelineParam(name='analyze-mode', value='local'),
    analyze_slice_column=dsl.PipelineParam(name='analyze-slice-column', value='trip_start_hour')):
    
    
    validation_output = '%s/{{workflow.name}}/validation' % output
    transform_output = '%s/{{workflow.name}}/transformed' % output
    training_output = '%s/{{workflow.name}}/train' % output
    analysis_output = '%s/{{workflow.name}}/analysis' % output
    prediction_output = '%s/{{workflow.name}}/predict' % output

    validation = dataflow_tf_data_validation_op(
        train, evaluation, column_names, key_columns, project,
        validation_mode, validation_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    preprocess = dataflow_tf_transform_op(
        train, evaluation, validation.outputs['schema'], project, preprocess_mode,
        preprocess_module, transform_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    training = tf_train_op(
        preprocess.output, validation.outputs['schema'], learning_rate, hidden_layer_size,
        steps, target, preprocess_module, training_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    analysis = dataflow_tf_model_analyze_op(
        training.output, evaluation, validation.outputs['schema'], project,
        analyze_mode, analyze_slice_column, analysis_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    prediction = dataflow_tf_predict_op(
        evaluation, validation.outputs['schema'], target, training.output,
        predict_mode, project, prediction_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    
    # The new deployer. Note that the DeployerOp interface is similar to the function "deploy_model".
    deploy = DeployerOp(
        gcp_project=project, model_name=model, version_name=version, runtime='1.9',
        model_path=training.output).apply(gcp.use_gcp_secret('user-gcp-sa'))

### Submit a new job

In [ ]:
compiler.Compiler().compile(my_taxi_cab_classification,  'my-tfx.zip')

run = client.run_pipeline(exp.id, 'my-tfx', 'my-tfx.zip',
                          params={'output': OUTPUT_DIR,
                                  'project': PROJECT_NAME,
                                  'model': DEPLOYER_MODEL,
                                  'version': DEPLOYER_VERSION_PROD})

result = client.wait_for_run_completion(run.id, timeout=600)

## Customize a step in Python2
Let's reuse the deploy_model function defined above. However, this time we will use python2 instead of the default python3.

In [ ]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
#TODO: demonstrate the python2 support in another sample.
DeployerOp = compiler.build_python_component(
    component_func=deploy_model,
    staging_gcs_path=OUTPUT_DIR,
    dependency=[kfp.compiler.VersionedDependency(name='google-api-python-client', version='1.7.0')],
    base_image='tensorflow/tensorflow:1.12.0',
    target_image=TARGET_IMAGE_TWO,
    python_version='python2')

### Modify the pipeline with the new deployer

In [ ]:
# My New Pipeline. It's almost the same as the original one with the last step deployer replaced.
@dsl.pipeline(
  name='TFX Taxi Cab Classification Pipeline Example',
  description='Example pipeline that does classification with model analysis based on a public BigQuery dataset.'
)
def my_taxi_cab_classification(
    output,
    project,
    model,
    version,
    column_names=dsl.PipelineParam(
        name='column-names',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/column-names.json'),
    key_columns=dsl.PipelineParam(name='key-columns', value='trip_start_timestamp'),
    train=dsl.PipelineParam(
        name='train',
        value=TRAIN_DATA),
    evaluation=dsl.PipelineParam(
        name='evaluation',
        value=EVAL_DATA),
    validation_mode=dsl.PipelineParam(name='validation-mode', value='local'),
    preprocess_mode=dsl.PipelineParam(name='preprocess-mode', value='local'),
    preprocess_module: dsl.PipelineParam=dsl.PipelineParam(
        name='preprocess-module',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/preprocessing.py'),
    target=dsl.PipelineParam(name='target', value='tips'),
    learning_rate=dsl.PipelineParam(name='learning-rate', value=0.1),
    hidden_layer_size=dsl.PipelineParam(name='hidden-layer-size', value=HIDDEN_LAYER_SIZE),
    steps=dsl.PipelineParam(name='steps', value=STEPS),
    predict_mode=dsl.PipelineParam(name='predict-mode', value='local'),
    analyze_mode=dsl.PipelineParam(name='analyze-mode', value='local'),
    analyze_slice_column=dsl.PipelineParam(name='analyze-slice-column', value='trip_start_hour')):
    
    
    validation_output = '%s/{{workflow.name}}/validation' % output
    transform_output = '%s/{{workflow.name}}/transformed' % output
    training_output = '%s/{{workflow.name}}/train' % output
    analysis_output = '%s/{{workflow.name}}/analysis' % output
    prediction_output = '%s/{{workflow.name}}/predict' % output

    validation = dataflow_tf_data_validation_op(
        train, evaluation, column_names, key_columns, project,
        validation_mode, validation_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    preprocess = dataflow_tf_transform_op(
        train, evaluation, validation.outputs['schema'], project, preprocess_mode,
        preprocess_module, transform_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    training = tf_train_op(
        preprocess.output, validation.outputs['schema'], learning_rate, hidden_layer_size,
        steps, target, preprocess_module, training_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    analysis = dataflow_tf_model_analyze_op(
        training.output, evaluation, validation.outputs['schema'], project,
        analyze_mode, analyze_slice_column, analysis_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    prediction = dataflow_tf_predict_op(
        evaluation, validation.outputs['schema'], target, training.output,
        predict_mode, project, prediction_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    
    # The new deployer. Note that the DeployerOp interface is similar to the function "deploy_model".
    deploy = DeployerOp(
        gcp_project=project, model_name=model, version_name=version, runtime='1.9',
        model_path=training.output).apply(gcp.use_gcp_secret('user-gcp-sa'))

### Submit a new job

In [ ]:
compiler.Compiler().compile(my_taxi_cab_classification,  'my-tfx-two.tar.gz')

run = client.run_pipeline(exp.id, 'my-tfx-two', 'my-tfx-two.tar.gz',
                          params={'output': OUTPUT_DIR,
                                  'project': PROJECT_NAME,
                                  'model': DEPLOYER_MODEL,
                                  'version': DEPLOYER_VERSION_PROD_TWO})

result = client.wait_for_run_completion(run.id, timeout=600)

## Clean up

In [17]:
# the step is only needed if you are using an in-cluster JupyterHub instance.
!gcloud auth activate-service-account --key-file /var/run/secrets/sa/user-gcp-sa.json


!gcloud ml-engine versions delete $DEPLOYER_VERSION_PROD --model $DEPLOYER_MODEL -q
!gcloud ml-engine versions delete $DEPLOYER_VERSION_PROD_TWO --model $DEPLOYER_MODEL -q
!gcloud ml-engine versions delete $DEPLOYER_VERSION_DEV --model $DEPLOYER_MODEL -q
!gcloud ml-engine models delete $DEPLOYER_MODEL -q

Activated service account credentials for: [kubeflow3-user@bradley-playground.iam.gserviceaccount.com]
Deleting version [prod]......done.                                             
Deleting version [dev]......done.                                              
Deleting model [notebook_tfx_taxi]...done.                                     
